<a href="https://colab.research.google.com/github/2019ht12169/Thesis/blob/main/simpleBertImpl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! python3.7 -m pip install tensorflow-gpu==1.15.0
import pandas as pd
from sklearn.model_selection import train_test_split

In [39]:
# check out what the data looks like before you get started
# look at the training data set
train_df = pd.read_csv('/content/gdrive/MyDrive/projects/data/dataFile_latest_train.csv')
train_df=train_df.drop(['review_date'],axis='columns')
print(train_df.head())

# look at the test data set
test_df = pd.read_csv('/content/gdrive/MyDrive/projects/data/dataFile_latest_test.csv')
test_df=test_df.drop(['review_date'],axis='columns')
print(test_df.head())

train_df = train_df.dropna()
test_df = test_df.dropna()

   label                                               text
0      1  with all the negative reviews i was hesitant t...
1      1  nice size radio and sounds good great little r...
2      1  good purchase nice compact speaker works great...
3      1  working nice since two years i have been using...
4      1  as advertised. no complaints. as advertised. n...
   label                                               text
0      1  five stars works well for my house where i con...
1      1  great for the price! i bought these to plug in...
2      1  great results this little antenna works very w...
3      1  great sound quality this set of headphones has...
4      1  everything i wanted in my new music device thi...


In [36]:
train_df['label'].astype(int)
test_df['label'].astype(int)

0         1
1         1
2         1
3         1
4         1
         ..
310062    0
310063    0
310064    0
310065    0
310066    0
Name: label, Length: 310054, dtype: int64

In [40]:
bert_df = pd.DataFrame({
    'id': range(len(train_df)),
    'label': train_df['label'],
    'alpha': ['q']*train_df.shape[0],
    'text': train_df['text'].replace(r'\n', ' ', regex=True)
})

train_bert_df, dev_bert_df = train_test_split(bert_df, test_size=0.2)

In [41]:
print(len(train_bert_df))
print(len(dev_bert_df))


992095
248024


In [42]:
test_bert_df = pd.DataFrame({
    'id': range(len(test_df)),
    'text': test_df['text'].replace(r'\n', ' ', regex=True)
})
test_bert_df.head()

,id,text
0,0,five stars works well for my house where i con...
1,1,great for the price! i bought these to plug in...
2,2,great results this little antenna works very w...
3,3,great sound quality this set of headphones has...
4,4,everything i wanted in my new music device thi...


In [43]:
train_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/train.tsv', sep='\t', index=False, header=False)
dev_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/dev.tsv', sep='\t', index=False, header=False)
test_bert_df.to_csv('/content/gdrive/MyDrive/projects/data/Data-BERT/test.tsv', sep='\t', index=False, header=False)

In [ ]:
! python /content/gdrive/MyDrive/projects/bert/run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=/content/gdrive/MyDrive/projects/data/Data-BERT --vocab_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_model.ckpt --max_seq_length=128 --train_batch_size=32 --learning_rate=2e-5 --num_train_epochs=3.0 --output_dir=/content/gdrive/MyDrive/projects/data/Data-BERT/model_output --do_lower_case=true save_checkpoints_steps 10000





W1115 12:52:52.748872 140688121284480 module_wrapper.py:139] From /content/gdrive/MyDrive/projects/bert/run_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1115 12:52:52.749058 140688121284480 module_wrapper.py:139] From /content/gdrive/MyDrive/projects/bert/run_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1115 12:52:52.749486 140688121284480 module_wrapper.py:139] From /content/gdrive/MyDrive/projects/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1115 12:52:52.751298 140688121284480 module_wrapper.py:139] From /content/gdrive/MyDrive/projects/bert/run_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/c

In [ ]:
!python /content/gdrive/MyDrive/projects/bert/run_classifier.py --task_name=cola --do_predict=true --data_dir=/content/gdrive/MyDrive/projects/data/Data-BERT --vocab_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/vocab.txt --bert_config_file=/content/gdrive/MyDrive/projects/uncased_L-12_H-768_A-12/bert_config.json --init_checkpoint=/content/gdrive/MyDrive/projects/model_output_latest/model.ckpt-43736 --max_seq_length=128 --output_dir=/content/gdrive/MyDrive/projects/data/Data-BERT/model_output/

In [ ]:
#read the original test data for the text and id
df_test = pd.read_csv('/content/gdrive/MyDrive/projects/data/test.tsv', sep='\t')

#read the results data for the probabilities
df_result = pd.read_csv('/content/gdrive/MyDrive/projects/model_output_latest/test_results.tsv', sep='\t', header=None)


In [ ]:
#create a new dataframe
df_map_result = pd.DataFrame({'id': df_test['id'],
    'text': df_test['text'],
    'label': df_result.idxmax(axis=1)})
#view sample rows of the newly created dataframe
df_map_result.sample(7)

,id,text,label
5,5,if you are looking for a good clingy plastic f...,0
2,2,ok,1
1,1,fantastic buy and a good plastic wrap even tho...,1
3,3,saran cling plus is kind of like most of the c...,1
0,0,good clinging,1
6,6,saran brand plastic wrap is normally pretty de...,0
4,4,the product description impled a pack of 12 th...,0


In [ ]:
countPos = 0
countNeg = 0
for value in df_map_result['label']:
    print(value)
    if value:
      countPos = countPos+1
    else:
      countNeg = countNeg+1

1
1
1
1
0
0
0
3
